#### Dataset Correction

In [25]:
import pandas as pd
import numpy as np
import math
import csv

columns = ['Flight No.', 'Timestamp', 'Altitude', 'Latitude', 'Longitude']
# Step 1: Read and preprocess the dataset
dataset = pd.read_csv("NA_11_Jun_29_2018_UTC11.CSV", sep = " ", names = columns)

dataset.to_csv("NA_11_Jun_29_2018_UTC11_Output.CSV", index = False)
# Extract the necessary columns: Flight No., Timestamp, Altitude, Latitude, and Longitude
print(dataset.head(10))

  Flight No.     Timestamp    Altitude   Latitude  Longitude
0      AA101  1.530270e+09  38975.0000  53.936440 -14.502541
1      AA151  1.530270e+09  35975.0000  49.129004  -4.044335
2      AA198  1.530270e+09      0.0000  39.792819 -72.778591
3      AA204  1.530270e+09      0.0000  40.222121 -73.325611
4      AA209  1.530270e+09  37975.0000  60.809745 -29.756008
5      AA221  1.530270e+09  32988.0208  59.381674 -21.598652
6       AA25  1.530270e+09  34000.0000  54.977373 -20.131168
7      AA258  1.530270e+09      0.0000  39.007541  -8.613116
8      AA291  1.530270e+09  34228.0290  52.983363 -23.482031
9       AA37  1.530270e+09  35982.3779  43.702034 -18.503210


#### Creating dataset for transmission link

In [26]:
data_trans = {"Mode k":[1, 2, 3, 4, 5, 6, 7], 
        "Mode Color":["Red", "Orange", "Yellow", "Green", "Blue", "Pink", "Purple"], 
        "Switching Threshold(km)":[500, 400, 300, 190, 90, 35, 5.56],
        "transmission Rate": [31.895, 43.505, 52.857, 63.970, 77.071, 93.854, 119.130]}
df_trans = pd.DataFrame(data_trans)
df_trans.to_csv("Transmission Link.csv", index = False)
print(df_trans)

   Mode k Mode Color  Switching Threshold(km)  transmission Rate
0       1        Red                   500.00             31.895
1       2     Orange                   400.00             43.505
2       3     Yellow                   300.00             52.857
3       4      Green                   190.00             63.970
4       5       Blue                    90.00             77.071
5       6       Pink                    35.00             93.854
6       7     Purple                     5.56            119.130


#### Conversion to 3D Cartesian Coordinate and finding the Distance Between them 

In [27]:
def calculate_distance(lat1, lon1, alt1, lat2, lon2, alt2):
    earth_radius = 6371  # Radius of the Earth in kilometers
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    equatorial_radius = 6378.137  # kilometers
    polar_radius = 6356.752  # kilometers
    eccentricity = math.sqrt(1 - (polar_radius ** 2) / (equatorial_radius ** 2))

    x1 = (equatorial_radius + alt1) * math.cos(lat1_rad) * math.cos(lon1_rad)
    y1 = (equatorial_radius + alt1) * math.cos(lat1_rad) * math.sin(lon1_rad)
    z1 = ((equatorial_radius * (1 - eccentricity ** 2)) + alt1) * math.sin(lat1_rad)


    x2 = (equatorial_radius + alt2) * math.cos(lat2_rad) * math.cos(lon2_rad)
    y2 = (equatorial_radius + alt2) * math.cos(lat2_rad) * math.sin(lon2_rad)
    z2 = ((equatorial_radius * (1 - eccentricity ** 2)) + alt2) * math.sin(lat2_rad)

    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
    return distance

#### Conversion to 3D Cartesian Coordinates for single value

In [28]:
def Convert_3D(latitude, longitude, altitude):
    
    earth_radius = 6371  # Radius of the Earth in kilometers
    latitude_rad = math.radians(latitude)
    longitude_rad = math.radians(longitude)

    equatorial_radius = 6378.137  # kilometers
    polar_radius = 6356.752  # kilometers
    eccentricity = math.sqrt(1 - (polar_radius ** 2) / (equatorial_radius ** 2))

    # Heathrow Airport coordinates to x, y, z coordinates
    x = (equatorial_radius + altitude) * math.cos(latitude_rad) * math.cos(longitude_rad)
    y = (equatorial_radius + altitude) * math.cos(latitude_rad) * math.sin(longitude_rad)
    z = ((equatorial_radius * (1 - eccentricity ** 2)) + altitude) * math.sin(latitude_rad)
    
    return x, y, z

#### To find transmission rate

##### Function to calculate the data transmission rate based on the distance between airplanes

In [29]:
def calculate_transmission_rate(distance):
    if 300 <= distance < 400:
        return 52.857
    elif 400 <= distance < 500:
        return 43.505
    elif 500 <= distance:
        return 31.895
    elif 190 <= distance < 300:
        return 63.970
    elif 90 <= distance < 190:
        return 77.071
    elif 35 <= distance < 90:
        return 93.854
    elif 5.56 <= distance < 35:
        return 119.130
    else:
        return 0

#### Single Objective Optimization

In [30]:
# Load and preprocess the dataset
airplanes = []
ground_stations = []

flight_name=input("Enter starting string code of flight: ")
with open('NA_11_Jun_29_2018_UTC11_Output.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row
    for row in reader:
        flight_no = row[0]
        altitude = float(row[2])
        latitude = float(row[3])
        longitude = float(row[4])

        # Convert latitude, longitude, and altitude to 3D Cartesian coordinates
        coordinates = Convert_3D(longitude, latitude, altitude)
        x = coordinates[0]
        y = coordinates[1]
        z = coordinates[2]

        if flight_no.startswith(flight_name):
            airplanes.append((flight_no, x, y, z))
        else:
            ground_stations.append((flight_no, x, y, z))

        # if flight_no.startswith('AA'):
        #     airplanes.append((flight_no, x, y, z))
        # else:
        #     ground_stations.append((flight_no, x, y, z))
        
        # if flight_no.startswith('BA'):
        #     airplanes.append((flight_no, x, y, z))
        # else:
        #     ground_stations.append((flight_no, x, y, z))
        
        # if flight_no.startswith('DA'):
        #     airplanes.append((flight_no, x, y, z))
        # else:
        #     ground_stations.append((flight_no, x, y, z))

        # if flight_no.startswith('LH'):
        #     airplanes.append((flight_no, x, y, z))
        # else:
        #     ground_stations.append((flight_no, x, y, z))

        # if flight_no.startswith('UA'):
        #     airplanes.append((flight_no, x, y, z))
        # else:
        #     ground_stations.append((flight_no, x, y, z))

In [31]:
def find_max_data_rate_routing_paths(airplanes, ground_stations):
    routing_paths = []

    for airplane in airplanes:
        airplane_id, x_airplane, y_airplane, z_airplane = airplane
        max_data_rate = 0.0
        max_data_rate_path = []

        for ground_station in ground_stations:
            ground_station_id, x_gs, y_gs, z_gs = ground_station

            # Calculate the distance between the airplane and ground station
            distance = calculate_distance(x_airplane, y_airplane, z_airplane, x_gs, y_gs, z_gs)

            # Calculate the data transmission rate for the link
            transmission_rate = calculate_transmission_rate(distance)

            if transmission_rate > max_data_rate:
                max_data_rate = transmission_rate
                max_data_rate_path = [(ground_station_id, max_data_rate)]
            # elif transmission_rate == max_data_rate:
            #     max_data_rate_path.append((ground_station_id, max_data_rate))


        routing_paths.append({'Airplane': airplane_id, 'Routing Path': max_data_rate_path, 'End-to-End Data Rate': max_data_rate})

    return routing_paths

# Call the function to find the routing paths with maximum data transmission rate
routing_paths = find_max_data_rate_routing_paths(airplanes, ground_stations)
all_routing_paths = find_max_data_rate_routing_paths(airplanes, ground_stations)


# Print and store the routing paths in a text file
with open('routing_paths.txt', 'w') as file:
    for path in all_routing_paths:
        file.write(str(path) + '\n')
        
# Print the routing paths and their respective data transmission rates
for path in routing_paths:
    print(path)

{'Airplane': 'AA37', 'Routing Path': [('AA101', 31.895)], 'End-to-End Data Rate': 31.895}


### Multiple objective optimisation

In [32]:
def find_optimal_routing_paths(airplanes, ground_stations):
    routing_paths = []

    for airplane in airplanes:
        airplane_id, x_airplane, y_airplane, z_airplane = airplane
        optimal_path = []
        max_data_rate = 0.0
        min_latency = float('inf')

        for ground_station in ground_stations:
            ground_station_id, x_gs, y_gs, z_gs = ground_station

            # Calculate the distance between the airplane and ground station
            distance = calculate_distance(x_airplane, y_airplane, z_airplane, x_gs, y_gs, z_gs)

            # Calculate the data transmission rate for the link
            transmission_rate = calculate_transmission_rate(distance)

            if transmission_rate == 0:
                latency = float('inf')
            else:
                # Calculate the latency for the link
                latency = distance / transmission_rate

            if transmission_rate > max_data_rate:
                max_data_rate = transmission_rate
                min_latency = latency
                optimal_path = [(ground_station_id, max_data_rate, min_latency)]
            elif transmission_rate == max_data_rate and latency < min_latency:
                min_latency = latency
                optimal_path = [(ground_station_id, max_data_rate, min_latency)]

        routing_paths.append({'Airplane': airplane_id, 'Optimal Path': optimal_path, 'End-to-End Data Rate': max_data_rate, 'End-to-End Latency': min_latency})

    return routing_paths

# Call the function to find the optimal routing paths with maximum data transmission rate and minimum latency
optimal_routing_paths = find_optimal_routing_paths(airplanes, ground_stations)

# Print and store the optimal routing paths in a text file
with open('optimal_routing_paths.txt', 'w') as file:
    for path in optimal_routing_paths:
        file.write(str(path) + '\n')
        print(path)


{'Airplane': 'AA37', 'Optimal Path': [('AA101', 31.895, 74.29584895383789)], 'End-to-End Data Rate': 31.895, 'End-to-End Latency': 74.29584895383789}
